In [13]:
!pip install python-dotenv
!pip install odp-sdk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 491.1 kB/s  0:00:14m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━ 24.9/29.6 MB 524.7 kB/s eta 0:00:09
Resuming download pyarrow-18.1.0-cp311-cp311-macosx_12_0_arm64.whl (24.9 MB/29.6 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.6/29.6 MB 6.4 MB/s  0:00:00 eta 0:00:01
  Attempting uninstall: validators
    Found existing installation: validators 0.35.0
    Uninstalling validators-0.35.0:
      Successfully uninstalled validators-0.35.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.0.0
    Uninstalling pyarrow-18.0.0:0m━━━━━━━━━━━━━━━━━━━━━━━━━━━  3/10 [pyarrow]
      Successfully uninstalled pyarrow-18.0.0━━━━━━━━━━━━━━━━━  3/10 [pyarrow]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10/10 [odp-sdk]9/10 [odp-sdk]aphy]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts

In [ ]:
import os 
api_key = os.environ.get('ODP_API_KEY')
print(api_key != None)

False
True


In [ ]:
#first test the api key with the sample from https://app.hubocean.earth/account

from odp.client import OdpClient
client = OdpClient(api_key=api_key)
dataset_name = "42cae68c-245a-4489-af48-6ad841b9ef65"
dataset = client.catalog.get(f"catalog.hubocean.io/dataset/{dataset_name}")
print(dataset)
tab = client.table_v2(dataset)
for row in tab.select().rows():
  print(row)
#for the moment this returns HTTP Error - 503: no available server

#ignore this and try features catalog...


/var/folders/4c/mxsj6f9d5_x8cjryn16fb1c40000gn/T/ipykernel_79839/3393960434.py:4: DeprecationWarning: catalog class is deprecated and will be removed in a future versions. Use catalog_v2 class instead.
  dataset = client.catalog.get(f"catalog.hubocean.io/dataset/{dataset_name}")


HTTPError: HTTP Error - 503: no available server


In [ ]:
#I've already used QGIS for this connection to preview some data, but qgis does not allow to discover all the option available.ValueError

# I'll use OWSlib 
import owslib
from owslib.ogcapi.features import Features
# connect to OGC Features endpoint with OWSLIB
_headers = {"Authorization": f"ApiKey sk_26baf4fa04be54136b2aa34b"}
odp_features_url = "https://api.hubocean.earth/api/features"
w = Features(odp_features_url, headers=_headers)

conformance = w.conformance()
conformance

{'conformsTo': ['http://www.opengis.net/spec/ogcapi-features-1/1.0/conf/core',
  'http://www.opengis.net/spec/ogcapi-features-1/1.0/conf/oas30',
  'http://www.opengis.net/spec/ogcapi-features-1/1.0/conf/html']}

So we know Features part 1 is implemented with this response
{'conformsTo': ['http://www.opengis.net/spec/ogcapi-features-1/1.0/conf/core',
  'http://www.opengis.net/spec/ogcapi-features-1/1.0/conf/oas30',
  'http://www.opengis.net/spec/ogcapi-features-1/1.0/conf/html']}

it should provide openAPI description.
maybe http://www.opengis.net/spec/ogcapi-features-1/1.0/conf/geojson
is missing lets check

In [31]:
w.api()

{'openapi': '3.0.3',
 'info': {'title': 'Ocean Data Platform OGC API – Features',
  'version': '0.1.0',
  'description': 'OGC API Features core.\n\n'},
 'servers': [{'url': 'https://api.hubocean.earth/api/features',
   'description': 'Default server'}],
 'paths': {'/': {'get': {'summary': 'Landing page',
    'responses': {'200': {'description': 'Landing page'}}}},
  '/conformance': {'get': {'summary': 'Conformance declaration',
    'responses': {'200': {'description': 'Conformance set'}}}},
  '/collections': {'get': {'summary': 'List collections',
    'responses': {'200': {'description': 'Collections list'}}}},
  '/collections/{collectionId}': {'get': {'summary': 'Describe collection',
    'parameters': [{'name': 'collectionId',
      'in': 'path',
      'required': True,
      'schema': {'type': 'string'}}],
    'responses': {'200': {'description': 'Collection metadata'}}}},
  '/collections/{collectionId}/items': {'get': {'summary': 'Fetch features',
    'parameters': [{'name': 'colle

Quicly looking looks useful. We know how can we query on the collections level
 - maybe /openapi itself could be added 
 - default limit of 25000 records is now low, check if it efficient enough


In [89]:
# Bounding box coordinates for UK and surrounding waters
# Format: [min_lon, min_lat, max_lon, max_lat]
uk_bbox = [-14.0, 49.0, 2.0, 62.0]

# Get collections with bbox parameter
collections = w.collections()
feature_collections = collections['collections']
def print_collection_info(collection):
    print(f"\nCollection ID: {collection['id']}")
    print(f"Title: {collection['title']}")
    print(f"Description: {collection.get('description', 'No description available')}")


def print_collections_info(collections):
    for collection in collections:
        print_collection_info(collection)
        

# Print the collections information
print_collections_info(feature_collections)


Collection ID: 2b535466-ebb3-4a2e-ba12-4f85c5025aa5
Title: [Swains] Benthic Habitats of American Samoa
Description:  Abstract: This project is a cooperative effort between the National Ocean Service, National Centers for Coastal Ocean Science, Center for Coastal Monitoring and Assessment, the University of Hawaii, BAE Systems Spectral Solutions and Analytical Laboratories of Hawaii, LLC. The goal of the work was to map the coral reef habitats of American Samoa, Guam and the Common Wealth of the Northern Mariana Islands by visual interpretation and manual delineation of IKONOS satellite imagery. A two tiered habitat classification system was tested and implemented in this work.  It integrates geomorphologic reef structure and biological cover into a single scheme and subsets each into detail.  It also includes thirteen zones.  Benthic features were mapped that covered an area of 71.5 square kilometers of which 10.56 were unconsolidated sediment and 60.94 were coral reef and hard bottom

In [91]:
#search in feature_collections the ones with the bbox intersecting the uk bbox
def bbox_intersects(bbox1, bbox2):
    return not (bbox1[2] < bbox2[0] or bbox1[0] > bbox2[2] or
                bbox1[3] < bbox2[1] or bbox1[1] > bbox2[3])
uk_collections = []
for collection in feature_collections:
    if 'extent' in collection and 'spatial' in collection['extent']:
        spatial_extent = collection['extent']['spatial']
        if 'bbox' in spatial_extent and len(spatial_extent['bbox']) > 0:
            collection_bbox = spatial_extent['bbox'][0]  # Assuming first bbox is the main one
            if bbox_intersects(collection_bbox, uk_bbox):
                uk_collections.append(collection)
print_collections_info(uk_collections)


Collection ID: 39d0a0c4-a9f2-42b8-aa6d-f777dd6374e7
Title: EMODnet Human Activities, Urban Waste Water Treatment Directive, Treatment Plants - Treatment Plants
Description: The datset on coastal Urban Waste Water Treatment Plant (UWWTP) was created in 2019 by AZTI for the European Marine Observation and Data Network (EMODnet). It is based on the dataset "Waterbase-UWWTD" (Urban Waste Water Treatment Directive – reported data) provided by the European Topic Centre on Inland, Coastal and Marine waters, and made available at https://www.eea.europa.eu/en/datahub/datahubitem-view/6244937d-1c2c-47f5-bdf1-33ca01ff1715. The datset is available for viewing and download on EMODnet - Human Activities web portal (https://emodnet.ec.europa.eu/en/human-activities). Based on data selected from the reporting of Member States as part of the UWWTD implementation, the dataset presents the available information (location, capacity and actual load treated, type of treatment, aggregated data on the perform

In [62]:
col_tides = next((c for c in uk_collections if c['id'] == '3975fa3a-52f2-4571-9ae5-9b4d8a39a65d'), None)
col_tides

{'id': '3975fa3a-52f2-4571-9ae5-9b4d8a39a65d',
 'title': 'Tides of Transparency - OBIS private sector classification - h3 resolution 2',
 'description': 'This dataset represents an analytical product of the report - Tides of Transparency: A First Mapping of Industrial Ocean Data Sharing by HUB Ocean and our partners. This data layer is a classification of all datapoints in OBIS as of August 2024 into which datapoints can be attributed to the private sector. The dataset is geographically binned into equal size areas based on the h3 index by Uber. The results can be found in three different levels: 2, 4 and 6 for performant access to different resolution levels. This dataset is at resolution 2 (5882 cells globally of approximately 86800km2).',
 'itemType': 'feature',
 'extent': {'spatial': {'bbox': [[-180.0, -90.0, 180.0, 90.0]],
   'crs': 'http://www.opengis.net/def/crs/OGC/1.3/CRS84'}},
 'links': [{'rel': 'items',
   'type': 'application/geo+json',
   'href': 'https://api.hubocean.eart

In [65]:
w.collection_queryables("3975fa3a-52f2-4571-9ae5-9b4d8a39a65d")

RuntimeError: {"request-id": "294a0b4855b7", "message": "Not Found"}

that would be good to have queryables but it is filtering challenge on the backend
But based on the openapi we know we can query by the bbox...

In [81]:
#get items around uk from the above collection
items = w.collection_items('3975fa3a-52f2-4571-9ae5-9b4d8a39a65d', bbox=uk_bbox, limit=3)
items

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [0.02523, 60.48359]},
   'properties': {'private_count': 177951,
    'h3_index': '8209a7fffffffff',
    'total_count': 1020686}},
  {'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [1.6606, 58.00035]},
   'properties': {'private_count': 349938,
    'h3_index': '8209b7fffffffff',
    'total_count': 1028082}},
  {'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [-11.60163, 52.67575]},
   'properties': {'private_count': 940,
    'h3_index': '821807fffffffff',
    'total_count': 682125}}]}

It may be useful to have next link for pagination

In [86]:
cables_items = w.collection_items('02ad43d3-d120-44b9-8c36-925479590ec0')
cables_items['features'][0]


{'type': 'Feature',
 'geometry': {'type': 'MultiLineString',
  'coordinates': [[[6.081704, 55.455452],
    [6.12879, 55.34018],
    [6.13022, 55.33817],
    [6.13173, 55.33674],
    [6.13385, 55.33529],
    [6.13603, 55.33428],
    [6.13878, 55.33349],
    [6.14526, 55.33266],
    [6.17737, 55.33268],
    [6.20609, 55.33266],
    [6.21028, 55.33267],
    [6.21843, 55.33267],
    [6.27566, 55.33267],
    [6.27956, 55.33223],
    [6.28295, 55.33143],
    [6.28661, 55.33001],
    [6.31938, 55.30797],
    [6.45126, 55.23385],
    [6.4963, 55.20848],
    [6.66091, 55.11516],
    [6.66412, 55.11397],
    [6.66753, 55.11308],
    [6.67147, 55.11261],
    [6.725, 55.11178],
    [6.77363, 55.111],
    [6.79443, 55.11038],
    [7.23311, 55.09649],
    [7.50339, 55.0871],
    [7.53041, 55.08616],
    [7.53791, 55.08592],
    [7.62708, 55.08285],
    [7.62729, 55.08285],
    [7.6301, 55.0824],
    [7.63239, 55.0817],
    [7.63487, 55.0805],
    [7.63737, 55.07862],
    [7.64651, 55.07159],
    [7.

it is great to see featuretype, any kind of link o the enum definition would be appreciated. idealy formal feature definition.